In [1]:
import os
import re 

from openai import OpenAI
# from mcts.settings.constants import *

LLAMA_MODEL = "llama3-8b-8192"
OPENAI_MODEL = "gpt-4o-mini"
GROQ_ENDPOINT = "https://api.groq.com/openai"
OPENAI_ENDPOINT = "https://api.openai.com"
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

openai_api_base = OPENAI_ENDPOINT + "/v1"

client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url=openai_api_base
)

In [14]:
def chat_completion_request_openai(prompt: str = ""):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        temperature=1.0,
        max_tokens=1500
    )
    if response.choices:
        return response.choices[0].message.content
    return None


def get_draft_answer_critique(question, draft_answer):
    prompt = (
        f"Question: {question}"
        f"Draft Answer: {draft_answer}"
        "Please critique the draft answer."
        "Carefully assess if the answer is correct of not and why."
        "Consider multiple ways of verifying if the answer is correct."
        "DO: Point out EVERY flaw and hold the draft answer to a very high standard."
        "DO: Provide specific recommendations to improve the answer."
        "DO: Think step by step."
        "DO NOT provide a revised answer"
    )
    return chat_completion_request_openai(prompt)


def get_improved_answer(question, draft_answer, critique):
    prompt = (
        f"Question: {question}"
        f"Draft Answer: {draft_answer}"
        f"Critique: {critique}"
        "Please improve the draft answer based on the critique following this format:"
        "Reasoning Process: <step-by-ste- reasoning process>\n"
        "Verification: <verification of the facts>\n"
        "Final Answer: <the improved and verified answer>\n"
    )
    return chat_completion_request_openai(prompt)

def rate_answer(question, answer):
    prompt = (
        f"Question: {question}"
        f"Answer: {answer}"
        "As an expert on this topic, please provide a detailed critique "
        "Provide ONLY a critique, not a suggested answer."
        "Then, rate the answer on a scale of 0 to 100."
        "The response should be in the following format:"
        "Critique: <detailed critique>\n"
        "Rating: <0 to 100 rating>\n"
    )
    response = chat_completion_request_openai(prompt)
    try: 
        match = re.search(r"Rating:\s+(\d+)", response)
        if match:
            rating = int(match.group(1))
            if rating > 95:
                rating = 95.0
            rating /= 100.0
        else:
            raise ValueError("Rating not found in response.")
    except Exception as e:
        print(f"Error extracting rating: {e}")
        print(f"Rating response was: {response}")
        rating = 0.0
    return rating

In [7]:
prompt = "return an array of strings where each array is a season"
response = chat_completion_request_openai(prompt)
print(response)

Here’s an array of strings where each element represents a season:

```javascript
const seasons = ["Spring", "Summer", "Fall", "Winter"];
```

If you have a specific programming language or format in mind, please let me know!


In [9]:
prompt = "What is the capital of France"
draft_answer = "The capital of France is Lyon"

critique = get_draft_answer_critique(prompt, draft_answer)
print(critique)

The draft answer contains a significant error. Here’s a step-by-step critique:

1. **Inaccuracy of Information**: The statement claims that the capital of France is Lyon, which is incorrect. The capital of France is Paris. This is a fundamental mistake, as it provides critical misinformation.

2. **Failure to Cite Sources**: The draft answer does not reference any sources or means of verification. A well-formed answer about a geographic fact should ideally provide a reliable source, such as an encyclopedia, government website, or educational resource. Citing reputable sources enhances credibility.

3. **Geographic Significance**: Even though Lyon is a major city in France, known for its historical and cultural importance, it does not serve as the capital. The answer fails to indicate this distinction and does not recognize the prominence of Paris as the nation's political, economic, and cultural center.

4. **Potential Misleading Implications**: By stating that Lyon is the capital, the

In [12]:
prompt = "What is the capital of France"
draft_answer = "The capital of France is Lyon"
critique = "That is just wrong"

improved_answer = get_improved_answer(prompt, draft_answer, critique)
print(improved_answer)

Reasoning Process: 
1. The draft answer states that the capital of France is Lyon. However, I need to recall the correct capital city of France.
2. France's capital is a well-known and widely recognized fact. I need to identify the capital city among the major cities in France.
3. In this case, Paris is generally recognized as the capital city of France, while Lyon is a major city but not the capital.

Verification:
- I will verify by checking multiple reliable sources or general knowledge about countries and their capitals. In geography, it is commonly known that Paris serves as the capital city of France.

Final Answer:
The capital of France is Paris.


In [15]:
question = "What is the capital of France"
answer = "The capital of France is Paris"
rating = rate_answer(question, answer)
print(f"Rating was: {rating}")

Rating was: 0.7


### Answer Improvement Rating Pipeline
For a given anser to a question:
1. Critique
2. Improve
3. Rate

In [10]:
seed_answers = [
    "I don't know the answer",
    "I'm not sure",
    "I can't say on that one"
]